# Import Necessary Libraries

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Import data

In [ ]:
# specify the path to your data 
# your directory should contain two image classes
# example for path.... C:\Users\project\project1\images
# images directory should contain two folders for two classes of image
#change path with your directory path
data_dir = r"path"

In [ ]:
# tf.keras.utils.image_dataset_from_directory() checks for folders and files in child directory
# its a common practice to clean your data before this step
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
# to avoid memory issues we divide data into batches 
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

### Data Visualisation

In [ ]:
#to show first 4 images in batch
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

### Normalise Data and Divide into training,validation and test data

In [ ]:
#Lamda function to normalise value to range 0-1 from 0-255
data = data.map(lambda x,y: (x/255, y))

In [ ]:
# set ratio for test,validate and test
#change value for desired ratio
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size

In [ ]:
#split data into train,validate and test
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
train

# Constructing a CNN

### import necessary modules

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Input

In [ ]:
#we use sequential model because we are constructing model for binary classification
model = Sequential()

In [ ]:
#input shape is shape of your input data
model.add(Input(shape=(256,256,3)))
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# adam optimizer takes learning rate as 0.01 as default value
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
# .summary() gives a summarized table for our CNN
model.summary()

### Training

In [ ]:
# to track the weights
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
#we use early stopping to stop the training if there is no significance change between each succesive epochs
# here we used val_loss as monitor;if val_loss doesn't show significant difference we stop the  training
# restore_best_weights restores best weights obtained during training to optimize model
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=3,          
    restore_best_weights=True
)

In [ ]:
# .fit() is used to train model,we provide training data,number of epochs,validation data and callbacks as parameters
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback, early_stopping_callback])

### Accuracy and Loss Visualisation 

In [ ]:
hist.history

In [ ]:
#plot to visualise  Accuracy V validation Accuracy
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
#plot to Visualise Loss V Validation Loss
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Testing

### Import necessary modules

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

### Test the model with test data

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# Testing the model with new Data

In [ ]:
# for multiple images in directory ; provide directory path
# run if you are testing with multiple image in directory
#change image_files[i] to select different images
img_path=r"dir_path"
if os.path.exists(img_path) and os.path.isdir(img_path):
    files = os.listdir(img_path)
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    image_path = os.path.join(img_path, image_files[0])
    print(image_path)

In [ ]:
#for single image;provide image path
image_path=r"image_path"

### Display and resize image

In [ ]:
img = cv2.imread(image_path)
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

### classify image

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if 1>= yhat >= 0.5: 
    print(f'Predicted class is class 1')
else:
    print(f'Predicted class is class 2')

# Save the model for future use

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#provide path to directory where model is to be saved
model.save(os.path.join(r'save_path','image_classifier.h5'))